Transpose Conv Operation Simple Explanation

In [2]:
import numpy as np

In [3]:
input = np.array([[4, 5, 8, 7], [1, 8, 8, 8], [3, 6, 6, 4], [6, 5, 7, 8]], np.float32)
print(input)

[[4. 5. 8. 7.]
 [1. 8. 8. 8.]
 [3. 6. 6. 4.]
 [6. 5. 7. 8.]]


In [4]:
kernel = np.array([[1, 4, 1], [1, 4, 3], [3, 3, 1]], np.float32)
print(kernel)

[[1. 4. 1.]
 [1. 4. 3.]
 [3. 3. 1.]]


In [5]:
output = np.zeros((2, 2), np.float32)
print(output)

[[0. 0.]
 [0. 0.]]


In [6]:
# conv operation
for i in range(2):
    for j in range(2):
        output[i][j] = np.sum(input[i:i+3, j:j+3] * kernel)
print(output)

[[122. 148.]
 [126. 134.]]


In [7]:
# construct a conv matrix that produces the same result as the kernel
# Padding the kernel to a matrix with the shape of (4 = 2x2, 16 = 4x4)
kernel_matrix = np.array([[1, 4, 1, 0, 1, 4, 3, 0, 3, 3, 1, 0, 0, 0, 0, 0], [0, 1, 4, 1, 0, 1, 4, 3, 0, 3, 3, 1, 0, 0, 0, 0], [0, 0, 0, 0, 1, 4, 1, 0, 1, 4, 3, 0, 3, 3, 1, 0], [0, 0, 0, 0, 0, 1, 4, 1, 0, 1, 4, 3, 0, 3, 3, 1]], np.float32)
print(kernel_matrix)

[[1. 4. 1. 0. 1. 4. 3. 0. 3. 3. 1. 0. 0. 0. 0. 0.]
 [0. 1. 4. 1. 0. 1. 4. 3. 0. 3. 3. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 4. 1. 0. 1. 4. 3. 0. 3. 3. 1. 0.]
 [0. 0. 0. 0. 0. 1. 4. 1. 0. 1. 4. 3. 0. 3. 3. 1.]]


In [8]:
# obtain the same result
output2 = np.matmul(kernel_matrix, input.reshape(16, 1)).reshape(2, 2)
print(output2)
print(output == output2)

[[122. 148.]
 [126. 134.]]
[[ True  True]
 [ True  True]]


The Conv Operation actually produces a 2x2=4x1 matrix from 4x4=16x1 matrix which can be implemented with (4, 16) matrix by matrix multiplication.
Thus, a (16, 4) matrix can also produces a 4x4=16x1 matrix from 2x2=4x1 matrix and this is the transpose convolution.